# Template for Lambda

In [1]:
#import modules here
import module



#define all helper functions here
def helper1():
    
    return

def helper2():
    
    return



def lambda_handler(event, context):
    
    
    #hard coded parameters here
    #Note : hard coded parameters must be passed in functions (functions can't access global variables)
    param1=1
    
    #get variables from the object "event" as a dictionary
    param2=event["param2"]
    
    data = event["datapoints"]
    
    result1 = helper1(param1)
    
    final_result = helper2(param1,param2,result1)
    
    return {"statusCode": 200}

    
    

SyntaxError: invalid syntax (Temp/ipykernel_7280/2839289373.py, line 2)

# Exemple

In [ ]:
import pandas as pd
import string
from datetime import datetime
import json
import re
from collections import OrderedDict
import traceback
import sys
import base64
import boto3



def message_cleaning_duplicate(message):
    messageToshow = ''
    messagetest = message.split(' ')
    messagelist = list(OrderedDict.fromkeys(message.split(' ')))
    for word in messagelist:
         messageToshow = messageToshow + ' ' + word
    return messageToshow


def add_element_to_list(time_logged_to_add,channel_to_add,username_message_to_add,messageToshow_to_add,list_to_add,hostchannel):
    tag_owner = 0
    if(('@' + hostchannel) in messageToshow_to_add):
                                    tag_owner = 1
    d = {
            'Date': time_logged_to_add,
            'Channel': channel_to_add,
            'Username': username_message_to_add,
            'Message': messageToshow_to_add,
            'Tag_owner':tag_owner
        }
    list_to_add.append(d)



def get_chat_dataframe(file,nickname,hostchannel):
    data = []

    lines = json.loads(file)

    username_message=''
    
    for line in lines:
        try:
            time_logged = line.split('—')[0].strip()
            if(time_logged != ''):
                time_logged = datetime.strptime(time_logged, '%Y-%m-%d_%H:%M:%S')

            username_message = line.split('—')[1:]
            infoInTheSameTime = username_message[0].split('\n\n')[1:]
            if(infoInTheSameTime):
                firstMultiInfo = 0
                for otherChat in infoInTheSameTime:
                    if(not firstMultiInfo):
                        messageToshow = username_message[0].split('\n\n')[0].split('#')[1:]
                        if(messageToshow):
                            channel = messageToshow[0].split(':')[0]
                            messageToshow = messageToshow[0].split(str(channel) + '' + ':')[1:][0]
                            username_message = username_message[0].split(':')[1:]
                            username_message = username_message[0].split('!')[0]
                            add_element_to_list(time_logged,channel,username_message,messageToshow,data,hostchannel)
                            firstMultiInfo = 1
                    messageToshow = ''
                    otherChatAtSameTime = otherChat.split(':')[1:]
                    username_message = otherChatAtSameTime[0].split('!')[0]
                    channel = otherChatAtSameTime[0].split('!')[1:]
                    for othermessage in otherChatAtSameTime[1:]:
                        messageToshow = messageToshow + '' + othermessage
                    if(channel and messageToshow and (nickname not in username_message)):
                        channel = channel[0].split('#')[1:][0]
                        add_element_to_list(time_logged,channel,username_message,messageToshow,data,hostchannel)

            else:
                messageToshow = username_message[0].split('#')[1:]
                channel = messageToshow[0].split(':')[0]
                messageToshow = messageToshow[0].split(str(channel) + '' + ':')[1:][0]
                username_message = username_message[0].split(':')[1:]
                username_message = username_message[0].split('!')[0]
                add_element_to_list(time_logged,channel,username_message,messageToshow,data,hostchannel)
        
        except Exception as e:
            if(len(username_message)==0):
                #print('WARNING : empty message')
                pass
            else:
                traceback.print_exc()
            pass
            
    return pd.DataFrame().from_records(data)




def lambda_handler(event, context):
    

    print(event)

    data=json.loads(event["body"])

    log = data["log"]
    hostchannel = data["channel"]
    nickname = data["nickname"]
   
    df = get_chat_dataframe(log,nickname,hostchannel)

    message_clean_duplicates = df['Message'].apply(message_cleaning_duplicate)
    df['clean_message_duplicates'] = message_clean_duplicates

    message_to_remove_list = []
    for message in df['Message']:
        
        if((len(message) <= 1) or
        (message.isnumeric() or (message.replace('-', '')).isnumeric()) or
        (('https' in message) or  ('.com' in message)) ):
            
            message_to_remove = df[df['Message'] == message]
            
            for same_messages in message_to_remove.values:
                
                d = {
                    'Channel': same_messages[1],
                    'Username': same_messages[2],
                    'Message': same_messages[3]
                }
                
                message_to_remove_list.append(d)
                
                if ((df['Message'] == message) & (df['Username'] == same_messages[2])).any():
                    df.drop(df[(df['Message'] == message) & (df['Username'] == same_messages[2])].index, inplace=True)


    
    lambda_client = boto3.client("lambda")
    response = lambda_client.invoke(
	FunctionName='pred',
	InvocationType='Event',
	LogType='None',
	Payload=json.dumps({"body" : df.to_json()}))
  
    return {"statusCode": 200}


    
    
   

Reference of a model deployment on AWS Labda : https://towardsdatascience.com/serverless-bert-with-huggingface-and-aws-lambda-625193c6cc04